# Lab 11: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [5]:
# Start the simulator
STOP_SIM()

# Start the plotter
STOP_PLOTTER()

2023-05-03 15:45:40,302 | ERROR    |: Simulator is not running
2023-05-03 15:45:40,304 | INFO     |: Plotter is stopped


In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
from localization_extras import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

2023-05-03 15:39:03,450 | INFO     |: Logger demo_notebook.log initialized.


TwoByTwoLayout(children=(Label(value='Simulator', layout=Layout(grid_area='top-left', width='80px')), HBox(chi…

qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=823.877,362.283 gbl=823.877,362.283 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-823.877,-362.283 last=-823.877,-362.283 Δ 823.877,362.283) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=846.239,341.275 gbl=846.239,341.275 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-846.239,-341.275 last=-846.239,-341.275 Δ 846.239,341.275) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=873.118,328.421 gbl=873.118,328.421 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-873.118,-328.421 last=-873.118,-328.421 Δ 873.118,328.421) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=898.197,322.379 gbl=898.197,322.379 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-898.197,-322.379 last=-898.197,-322.379 Δ 898.197,322.379) : no target window
qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=921.641,320.182 gbl=921.641,320.182 Released ellipse=(1x

In [2]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = VirtualRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a VirtualRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2023-05-03 15:39:07,253 | INFO     |:  | Number of observations per grid cell: 18
2023-05-03 15:39:07,255 | INFO     |:  | Precaching Views...


/Users/tiffanyguo/Desktop/ECE4160-code/ble_robot-1.1/FastRobots-sim-release-release-mirror/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2023-05-03 15:39:12,721 | INFO     |:  | Precaching Time: 5.464 secs
2023-05-03 15:39:12,723 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-03 15:39:12,724 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run the Bayes Filter
The cells below utilizes the member functions of class **Localization** (defined in [localization_extras.py](../localization_extras.py)) in each iteration of the Bayes filter algorithm to localize the robot in the grid map. <br>

In [3]:
# Reset Robot and Plots
robot.reset()
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.print_update_stats(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

2023-05-03 15:39:13,179 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-05-03 15:39:13,180 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
2023-05-03 15:39:17,112 | INFO     |: Update Step
2023-05-03 15:39:17,116 | INFO     |:      | Update Time: 0.003 secs
2023-05-03 15:39:17,117 | INFO     |: ---------- UPDATE STATS -----------
2023-05-03 15:39:17,126 | INFO     |: GT index      : (6, 4, 9)
2023-05-03 15:39:17,127 | INFO     |: Bel index     : (5, 4, 9) with prob = 0.8559505
2023-05-03 15:39:17,128 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2023-05-03 15:39:17,129 | INFO     |: GT            : (0.000, 0.000, 360.000)
2023-05-03 15:39:17,130 | INFO     |: Belief        : (0.000, 0.000, 10.000)
2023-05-03 15:39:17,132 | INFO     |: POS ERROR     : (-0.000, -0.000, 350.000)
2023-05-03 15:39:17,134 | INFO     |: ---------- UPDATE STATS -----------


In [4]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    loc.prediction_step(current_odom, prev_odom)
    loc.print_prediction_stats(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    loc.get_observation_data()
    
    # Update Step
    loc.update_step()
    loc.print_update_stats(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")



----------------- 0 -----------------


qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=844.512,188.481 gbl=844.512,188.481 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-844.512,-188.481 last=-844.512,-188.481 Δ 844.512,188.481) : no target window


2023-05-03 15:39:20,425 | INFO     |: Prediction Step
2023-05-03 15:39:20,546 | INFO     |:  | Prediction Time: 0.119 secs
2023-05-03 15:39:20,547 | INFO     |: ---------- PREDICTION STATS -----------
2023-05-03 15:39:20,557 | INFO     |: GT index         : (6, 3, 7)
2023-05-03 15:39:20,559 | INFO     |: Prior Bel index  : (5, 5, 8) with prob = 0.1107539
2023-05-03 15:39:20,561 | INFO     |: POS ERROR        : (0.287, -0.391, -29.534)
2023-05-03 15:39:20,565 | INFO     |: ---------- PREDICTION STATS -----------


qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=777.821,302.767 gbl=777.821,302.767 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-777.821,-302.767 last=-777.821,-302.767 Δ 777.821,302.767) : no target window


2023-05-03 15:39:23,490 | INFO     |: Update Step
2023-05-03 15:39:23,494 | INFO     |:      | Update Time: 0.003 secs
2023-05-03 15:39:23,496 | INFO     |: ---------- UPDATE STATS -----------
2023-05-03 15:39:23,502 | INFO     |: GT index      : (6, 3, 7)
2023-05-03 15:39:23,505 | INFO     |: Bel index     : (6, 4, 6) with prob = 1.0
2023-05-03 15:39:23,507 | INFO     |: Bel_bar prob at index = 0.001316363471556414
2023-05-03 15:39:23,509 | INFO     |: GT            : (0.287, -0.086, 320.466)
2023-05-03 15:39:23,511 | INFO     |: Belief        : (0.305, 0.000, -50.000)
2023-05-03 15:39:23,513 | INFO     |: POS ERROR     : (-0.017, -0.086, 370.466)
2023-05-03 15:39:23,515 | INFO     |: ---------- UPDATE STATS -----------
-------------------------------------


----------------- 1 -----------------
2023-05-03 15:39:25,551 | INFO     |: Prediction Step
2023-05-03 15:39:25,607 | INFO     |:  | Prediction Time: 0.055 secs
2023-05-03 15:39:25,608 | INFO     |: ---------- PREDICTION STATS --

...

2023-05-03 15:40:38,697 | INFO     |: Prediction Step
2023-05-03 15:40:38,750 | INFO     |:  | Prediction Time: 0.052 secs
2023-05-03 15:40:38,752 | INFO     |: ---------- PREDICTION STATS -----------
2023-05-03 15:40:38,765 | INFO     |: GT index         : (4, 3, 1)
2023-05-03 15:40:38,766 | INFO     |: Prior Bel index  : (4, 3, 0) with prob = 0.1391339
2023-05-03 15:40:38,768 | INFO     |: POS ERROR        : (-0.016, 0.044, 5418.255)
2023-05-03 15:40:38,770 | INFO     |: ---------- PREDICTION STATS -----------


Traceback (most recent call last):
  File "/Users/tiffanyguo/Desktop/ECE4160-code/ble_robot-1.1/FastRobots-sim-release-release-mirror/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
AttributeError: type object 'Qt' has no attribute 'Key_Escape'
Traceback (most recent call last):
  File "/Users/tiffanyguo/Desktop/ECE4160-code/ble_robot-1.1/FastRobots-sim-release-release-mirror/src/plotter.py", line 263, in keyPressEvent
    if event.key() == Qt.Key_Escape:
AttributeError: type object 'Qt' has no attribute 'Key_Escape'


2023-05-03 15:40:41,729 | INFO     |: Update Step
2023-05-03 15:40:41,735 | INFO     |:      | Update Time: 0.005 secs
2023-05-03 15:40:41,736 | INFO     |: ---------- UPDATE STATS -----------
2023-05-03 15:40:41,739 | INFO     |: GT index      : (4, 3, 1)
2023-05-03 15:40:41,742 | INFO     |: Bel index     : (4, 3, 1) with prob = 1.0
2023-05-03 15:40:41,745 | INFO     |: Bel_bar prob at index = 0.02718916654055326
2023-05-03 15:40:41,747 | INFO     |: GT            : (-0.320, -0.261, 5608.255)
2023-05-03 15:40:41,750 | INFO     |: Belief        : (-0.305, -0.305, -150.000)
2023-05-03 15:40:41,751 | INFO     |: POS ERROR     : (-0.016, 0.044, 5758.255)
2023-05-03 15:40:41,754 | INFO     |: ---------- UPDATE STATS -----------
-------------------------------------


----------------- 15 -----------------
2023-05-03 15:40:44,810 | INFO     |: Prediction Step
2023-05-03 15:40:44,865 | INFO     |:  | Prediction Time: 0.054 secs
2023-05-03 15:40:44,866 | INFO     |: ---------- PREDICTION STA

Traceback (most recent call last):
  File "/Users/tiffanyguo/Desktop/ECE4160-code/ble_robot-1.1/FastRobots-sim-release-release-mirror/src/plotter.py", line 252, in closeEvent
    result = QtGui.QMessageBox.question(self,
AttributeError: module 'pyqtgraph.Qt.QtGui' has no attribute 'QMessageBox'


Loading Flatland...
Initializing pygame framework...
